# Задача: Контекстный менеджер для работы с файлами

## Описание
Реализуйте контекстный менеджер `FileManager`, который управляет открытием и закрытием файлов. Контекстный менеджер должен автоматически закрывать файл при выходе из блока `with`, даже если внутри блока возникает исключение.

Кроме того, при работе с файлом необходимо проверять, что файл закрыт после выхода из контекста и открыт в его пределах.

## Требования
1. Реализуйте класс `FileManager`, который принимает два параметра:
   - `filepath`: путь к файлу.
   - `mode`: режим открытия файла (например, 'w' для записи).
   
2. Контекстный менеджер должен:
   - Открывать файл при входе в блок `with` (метод `__enter__`).
   - Закрывать файл при выходе из блока `with`, независимо от того, произошло ли исключение (метод `__exit__`).
   - Выводить сообщения в консоль при открытии и закрытии файла (например, "Файл открыт" и "Файл закрыт").


In [2]:
class FileManager:
    pass

In [3]:
import os

import pytest


@pytest.fixture
def filepath():
    file = "test_file.txt"

    yield file

    os.remove(file)


# Тест успешного открытия и закрытия файла
def test_file_manager_write(filepath):
    with FileManager(filepath, "w") as f:
        f.write("Hello, pytest!")

        assert not f.closed

    assert f.closed

    # Проверяем, что файл был создан и содержит корректные данные
    assert os.path.exists(filepath)
    with open(filepath, "r") as f:
        content = f.read()
        assert content == "Hello, pytest!"

    assert f.closed


# Тест на проверку корректной обработки исключений
def test_file_manager_handles_exception(filepath):
    try:
        with FileManager(filepath, "w") as f:
            f.write("Some data")
            raise ValueError("Some error")
    except ValueError as e:
        assert str(e) == "Some error"

    # Проверяем, что файл был создан и содержит данные до возникновения исключения
    assert os.path.exists(filepath)
    with open(filepath, "r") as f:
        content = f.read()
        assert content == "Some data"


ModuleNotFoundError: No module named 'pytest'

# Задача: Контекстный менеджер с выборочным заглушением исключений

## Описание
Реализуйте контекстный менеджер `SuppressExceptions`, который заглушает исключения только определенных типов, передаваемых при инициализации. Если возникает исключение, которое не входит в указанные типы, оно должно быть выброшено и не подавляться.

Контекстный менеджер должен выводить информацию о типе и сообщении заглушенного исключения в консоль. Исключения, которые не попадают в список заглушаемых, должны быть выброшены и обработаны обычным образом.

## Требования
1. Реализуйте класс `SuppressExceptions`, который принимает параметр:
   - `suppress_types`: кортеж типов исключений, которые нужно заглушать (по умолчанию — все исключения).

2. Контекстный менеджер должен:
   - Заглушать только те исключения, которые указаны в `suppress_types`.
   - Выводить тип и сообщение заглушенного исключения в консоль.
   - Не заглушать исключения, которые не указаны в `suppress_types`.

3. Необходимо самим покрыть класс тестами.


In [ ]:
class SuppressExceptions:
    pass


# Задача: Управление транзакциями в симуляторе изменений

## Описание
Реализуйте класс `Transaction`, который представляет собой симуляцию транзакции над коллекцией данных. Транзакция должна быть обработана с помощью контекстного менеджера: при успешном завершении транзакции изменения фиксируются (коммит), а в случае ошибки откатываются (роллбек). Транзакция поддерживает изменения в виде добавления, обновления и удаления данных из коллекции.

### Основные функциональные требования:
- Добавление, обновление и удаление элементов коллекции.
- Автоматический коммит (фиксация изменений) при успешном завершении блока `with`.
- Откат всех изменений (роллбэк) в случае возникновения ошибки внутри блока `with`.
- Кастомные исключения для ошибок транзакций:
  - `TransactionError`: если транзакция уже завершена и не может быть использована.
  - `InvalidTransactionOperation`: если совершается некорректная операция в транзакции (например, удаление несуществующего элемента).

### Поведение контекстного менеджера:
- Если блок `with` завершился успешно (без исключений), контекстный менеджер автоматически вызывает `commit()`.
- Если внутри блока возникло исключение, контекстный менеджер вызывает `rollback()`, откатывая все изменения.

## Требования
1. Реализуйте класс `Transaction`, который:
   - Содержит коллекцию данных (например, список).
   - Реализует контекстный менеджер для управления транзакцией: при завершении блока `with` должен либо коммитить (фиксировать изменения), либо откатывать (отменять изменения в случае ошибки).
   - Поддерживает операции:
     - `add(item)`: добавление элемента в коллекцию.
     - `update(index, new_item)`: обновление элемента по индексу.
     - `delete(index)`: удаление элемента по индексу.

2. Реализуйте кастомные исключения:
   - `TransactionError`: выбрасывается, если транзакция уже завершена и не может быть использована.
   - `InvalidTransactionOperation`: выбрасывается при попытке выполнения некорректной операции (например, удаление несуществующего элемента).

## Пример использования:

```python
data = ["apple", "banana", "cherry"]

try:
    with Transaction(data) as transaction:
        transaction.add("date")
        transaction.update(1, "blueberry")
        transaction.delete(2)
        # Ошибка для демонстрации отката
        raise ValueError("Возникла ошибка в транзакции")
except ValueError as e:
    print("Транзакция откатана:", data)

# Ожидаемый вывод: данные не изменились, так как произошел откат
# Транзакция откатана: ['apple', 'banana', 'cherry']

with Transaction(data) as transaction:
    transaction.add("date")
    transaction.update(1, "blueberry")
    transaction.delete(2)

print("Измененные данные:", data)
# Ожидаемый вывод: Измененные данные: ['apple', 'blueberry', 'date']

```

In [4]:
class TransactionError(Exception):
    pass


class InvalidTransactionOperation(Exception):
    pass


class Transaction:
    pass
